# 使用 Tensorflow 实现 卷积神经网络

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# parameters

learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# network parameters

n_input = 784
n_classes = 10
dropout = 0.75  # probability to keep units

# input 

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [4]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding="SAME")
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k=2):
    # maxpool2d wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

In [6]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # convolution layer
    conv1 = conv2d(x, weights["wc1"], biases["bc1"])
    conv1 = maxpool2d(conv1, k=2)
    
    conv2 = conv2d(conv1, weights["wc2"], biases["bc2"])
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights["wd1"].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights["wd1"]), biases["bd1"])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights["out"]), biases["out"])
    return out

In [7]:
weights = {
    "wc1": tf.Variable(tf.random_normal([5, 5, 1, 32])),
    "wc2": tf.Variable(tf.random_normal([5, 5, 32, 64])),
    "wd1": tf.Variable(tf.random_normal([7 * 7 * 64, 1024])),
    "out": tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    "bc1": tf.Variable(tf.random_normal([32])),
    "bc2": tf.Variable(tf.random_normal([64])),
    "bd1": tf.Variable(tf.random_normal([1024])),
    "out": tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
pred = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [10]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(optimizer, feed_dict={
            x: batch_x,
            y: batch_y,
            keep_prob: dropout
        })
        
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })
            print("Iter: {:d}, Minibatch Loss={:.6f}, Training Accuracy={:.5f}".format(step*batch_size, loss, acc))
        step += 1
    print("Optimization Finished!")
    print("Testing Accuracy: {:.5f}".format(sess.run(accuracy, feed_dict={
        x: mnist.test.images[:256],
        y: mnist.test.labels[:256],
        keep_prob: 1.
    })))

Iter: 1280, Minibatch Loss=20948.316406, Training Accuracy=0.17969
Iter: 2560, Minibatch Loss=10720.512695, Training Accuracy=0.48438
Iter: 3840, Minibatch Loss=7876.099121, Training Accuracy=0.60156
Iter: 5120, Minibatch Loss=4711.554688, Training Accuracy=0.74219
Iter: 6400, Minibatch Loss=4238.761230, Training Accuracy=0.74219
Iter: 7680, Minibatch Loss=5195.278320, Training Accuracy=0.75781
Iter: 8960, Minibatch Loss=2770.641113, Training Accuracy=0.79688
Iter: 10240, Minibatch Loss=2942.683105, Training Accuracy=0.83594
Iter: 11520, Minibatch Loss=1038.053101, Training Accuracy=0.91406
Iter: 12800, Minibatch Loss=3103.335693, Training Accuracy=0.83594
Iter: 14080, Minibatch Loss=911.368713, Training Accuracy=0.90625
Iter: 15360, Minibatch Loss=1341.486694, Training Accuracy=0.90625
Iter: 16640, Minibatch Loss=1867.321045, Training Accuracy=0.90625
Iter: 17920, Minibatch Loss=925.473328, Training Accuracy=0.93750
Iter: 19200, Minibatch Loss=1213.639893, Training Accuracy=0.91406
It